In [17]:
import pandas as pd
ml_df = pd.read_csv("datasets/full_dataset.csv")
ml_df.head()

,Course Number,CRN,Type,Section,Time,Days,Location,Instructor
0,CS 124,74477,Lecture,AL1,ARRANGED,-,-,"Challen, G"
1,CS 124,75242,Lecture,AL2,ARRANGED,-,-,"Challen, G"
2,CS 124,74481,Lab/Discussion,QBA,10:00 AM - 10:50 AM,T,4025 - Campus Instructional Facility,"Challen, G"
3,CS 124,74482,Lab/Discussion,QBB,10:00 AM - 10:50 AM,T,4029 - Campus Instructional Facility,"Challen, G"
4,CS 124,74490,Lab/Discussion,QBC,11:00 AM - 11:50 AM,T,4025 - Campus Instructional Facility,"Challen, G"


In [18]:
df['Instructor_Last'] = df['Instructor'].str.split(',').str[0].str.strip()
ml_df['Instructor_Last'] = ml_df['Instructor'].str.split(',').str[0].str.strip()

In [19]:
df = pd.read_csv("datasets/viz1.csv")  # Replace with your path if needed
df['Course Number'] = df['Subject'] + ' ' + df['Number'].astype(str)
df['Instructor'] = df['Primary Instructor']

In [20]:
ml_df.loc[ml_df['Course Number'] == 'STAT 107', 'Instructor_Last'] = 'Flanagan'
df.loc[df['Course Number'] == 'STAT 107', 'Instructor_Last'] = 'Flanagan'


In [21]:
df['Instructor_Last'] = df['Instructor'].str.split(',').str[0].str.strip()
ml_df['Instructor_Last'] = ml_df['Instructor'].str.split(',').str[0].str.strip()
ml_df.loc[ml_df['Course Number'] == 'STAT 107', 'Instructor_Last'] = 'Flanagan'

# Also ensure 'Course Number' matches in both (you used 'Course Number' in merge key)
# If they differ in naming (like 'Number' in df), rename accordingly:
# df.rename(columns={'Number': 'Course Number'}, inplace=True)

# Merge using course number and instructor last name
df_com = pd.merge(df, ml_df, on=['Course Number', 'Instructor_Last'])

In [22]:
df_complete = df_com.drop(['Instructor_x', 'Instructor_Last', 'Location', 'CRN', 'Instructor_y', 'Section'], axis=1, errors='ignore')
df_complete['Primary Instructor'] = df_complete['Primary Instructor'].str.replace(
    r'^([^,]+,\s+\w+)\s+\w\.?$', r'\1', regex=True
)
df_complete.head()

,Unnamed: 0,Primary Instructor,Course Title,Subject,Number,Overall_GPA,Total_Students,A_range,B_range,C_range,D_range,F_range,YearTerm,Course Number,Type,Time,Days
0,24,"Bronski, Jared",Calculus II,MATH,231,2.994208,3190,35.931318,38.089262,19.564219,4.57907,1.836130,13,MATH 231,Lecture,09:00 AM - 09:50 AM,MWF
1,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,2.48599,3.361178,8,CS 124,Lecture,ARRANGED,-
2,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,2.48599,3.361178,8,CS 124,Lecture,ARRANGED,-
3,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,2.48599,3.361178,8,CS 124,Lab/Discussion,10:00 AM - 10:50 AM,T
4,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,2.48599,3.361178,8,CS 124,Lab/Discussion,10:00 AM - 10:50 AM,T


In [23]:
from datetime import time

# Define bounds
start_bound = time(10, 0)
end_bound = time(14, 0)

# Create Start_Time column
def extract_start_time(row):
    if pd.isna(row) or row.strip().upper() == 'ARRANGED':
        return 'ARRANGED'
    try:
        start_str = row.split('-')[0].strip()
        return pd.to_datetime(start_str, format='%I:%M %p').time()
    except:
        return None

df_complete['Start_Time'] = df_complete['Time'].apply(extract_start_time)

# Create Time_Category column
def classify_time(t):
    if t == 'ARRANGED':
        return 1
    elif isinstance(t, time):
        return 1 if start_bound <= t <= end_bound else -1
    else:
        return -1  # default fallback

df_complete['Time_Category'] = df_complete['Start_Time'].apply(classify_time)


In [24]:
def sentiment_function(row):
    course = row['Course Number']
    if course == 'STAT 107':
        return 0.9628
    elif course == 'MATH 231':
        return 0.9852
    elif course == 'MATH 257':
        return 0.872
    elif course == 'CS 124':
        return 0.9711
    elif course == 'ECON 102':
        return 0.8316
    elif course == 'IS 445':
        return 0.9568
    else:
        return None  # or some default score

# Apply to each row
df_complete['Sentiment Score'] = df_complete.apply(sentiment_function, axis=1)


df_complete['synthetic_rating'] = ((
    df_complete['Overall_GPA'] * 2.0 +
    df_complete['A_range'] * 0.5 -
    df_complete['F_range'] * 0.8 -
    df_complete['C_range'] * 0.2 +
    df_complete['YearTerm'] * 0.05 +
    df_complete['Time_Category'] +
    df_complete['Sentiment Score'] * 2
)/(64.55)) * 100
df_complete.head()


,Unnamed: 0,Primary Instructor,Course Title,Subject,Number,Overall_GPA,Total_Students,A_range,B_range,C_range,...,F_range,YearTerm,Course Number,Type,Time,Days,Start_Time,Time_Category,Sentiment Score,synthetic_rating
0,24,"Bronski, Jared",Calculus II,MATH,231,2.994208,3190,35.931318,38.089262,19.564219,...,1.836130,13,MATH 231,Lecture,09:00 AM - 09:50 AM,MWF,09:00:00,-1,0.9852,31.282304
1,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,...,3.361178,8,CS 124,Lecture,ARRANGED,-,ARRANGED,1,0.9711,71.040543
2,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,...,3.361178,8,CS 124,Lecture,ARRANGED,-,ARRANGED,1,0.9711,71.040543
3,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,...,3.361178,8,CS 124,Lab/Discussion,10:00 AM - 10:50 AM,T,10:00:00,1,0.9711,71.040543
4,30,"Challen, Geoffrey",Intro to Computer Science I,CS,124,3.564427,6608,77.892957,11.900277,4.359598,...,3.361178,8,CS 124,Lab/Discussion,10:00 AM - 10:50 AM,T,10:00:00,1,0.9711,71.040543


In [25]:
df_complete.to_csv("datasets/final_dataset_for_course_catalog_and_score.csv")


In [26]:
df_complete['Course Number'].unique()

array(['MATH 231', 'CS 124', 'MATH 257', 'ECON 102', 'STAT 107', 'IS 445'],
      dtype=object)